In [ ]:
import itertools
import math
import re
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.rcParams["figure.figsize"] = (16, 6)
# plt.rcParams["figure.autolayout"] = True # equivalent to .tight_layout(); breaks some legends
plt.rcParams["figure.constrained_layout.use"] = True  # Experimental

sns.set_style("white")
sns.set_context("notebook")

In [ ]:
## ESCs and EXPs to plot
# order here determines order in plots
selected_exps = ["DE", "ES", "AR"]
selected_escs = [
    "hvdc-to-h2",
    "pipeline-h2",
    "shipping-lh2",
    "shipping-lohc",
    "pipeline-ch4",
    "shipping-lch4",
    "shipping-lnh3",
    "shipping-lnh3-to-h2",
    "shipping-meoh",
    "shipping-ftfuel",
]

In [ ]:
## Select and prepare data of interest
df = pd.read_csv(snakemake.input[0], sep=";")
df = df.loc[
    (df["scenario"] == snakemake.wildcards["scenario"])
    & (df["importer"] == "DE")
    & (df["year"] == int(snakemake.wildcards["year"]))
    & (df["esc"].isin(selected_escs))
    & (df["exporter"].isin(selected_exps))
]

# Used later, assume demand for all ESCs is identical
demand = df.query("subcategory == 'Total demand'")["value"].unique()
assert demand.size == 1, "Demand must be identical for all ESCs"
demand = demand.item()

# Only cost data required for plotting
df = df.loc[df["category"] == "cost"]

# Prepare correct data format
df = df.pivot(index=["esc", "exporter"], columns="subcategory", values="value")

In [ ]:
# Aggregate columns and nicer names
# Order here also directs the order in the plot later
columns_to_aggregate = {
    "PV | wind": "windonshore|windoffshore|pvplant",
    "water desalination & storage": "(seawater desalination|clean water tank storage)",
    "battery storage": "battery",
    "electrolysis": "electrolysis",
    #'heat pump':'industrial heat pump medium temperature',
    #'DAC':'direct air capture',
    "DAC incl. heat pump | LOHC chemical": (
        "(direct air capture)|(industrial heat pump medium temperature)|"
        "^((LOHC chemical)|(LOHC treatment)|(LOHC \(loaded\) (un)?loading)|(LOHC \(used\) (un)?loading))"
    ),
    "storage H2 (g) incl. compression": "(hydrogen demand buffer)|(hydrogen \(g\) demand buffer)|(hydrogen storage)|(H2 storage)",  # Storage + buffer + compressors
    "storage CO2 (l) incl. conversion": "CO2 (liquefaction|evaporation|storage tank)",
    "storage CH4 (g) incl. compression": "(CH4 storage incl. compression)|(CH4 storage)|(methane storage)|(methane \(g\) demand buffer)",
    #'storage hydrocarbon product':'(((FT fuel)|methanol) demand buffer)|(((FT fuel)|MeOH) storing)|(General liquid hydrocarbon storage)',
    #'storage LOHC':'LOHC (unloaded|loaded) DBT storage',
    #'storage CH4':'(CH4 storage)|(methane storage)|(methane \(g\) demand buffer)', # CH4 storage and compressor cost
    #'storage NH3':'NH3 ((evaporation|liquefaction)|(\(l\) storage tank incl. liquefaction))|ammonia \(g\) demand buffer',
    "storage H2(l) | LOHC | CH4(l) | NH3(l) | MeOH | FT fuel": (
        "(H2 \(l\) storage tank)|"
        "(FT fuel demand buffer)|"
        "(methanol demand buffer)|"
        "(FT fuel storing)|"
        "(MeOH storing)|"
        "(General liquid hydrocarbon storage)|"
        "(LOHC (unloaded|loaded) DBT storage)|"
        "(LNG storage tank)|(CH4 \(l\) storing and unstoring)|"
        "NH3 ((evaporation|liquefaction)|(\(l\) storage tank incl. liquefaction))|ammonia \(g\) demand buffer"
    ),
    #'Methanation':'methanation',
    #'Methanol synthesis': 'methanolisation',
    #'Ammonia synthesis':'(air separation unit|Haber-Bosch)',
    #'Fischer-Tropsch':'Fischer-Tropsch',
    "synthesis CH4 | NH3| MeOH | FT fuel": (
        "(methanation)|"
        "(methanolisation)|"
        "(air separation unit)|(Haber-Bosch)|"
        "(Fischer-Tropsch)"
    ),
    #'de-/hydrogenation LOHC': 'LOHC (de)?hydrogenation',
    #'Ammonia to Hydrogen cracking': 'Ammonia cracker',
    "H2(l) & CH4(l) liq. & regas. | LOHC (de-)hydrog. | NH3 crack.": (
        "(LOHC (de)?hydrogenation)|"
        "(Ammonia cracker)|"
        "H2 ((evaporation|liquefaction))|"
        "CH4 ((evaporation|liquefaction))"
    ),
    #'HVDC': 'HVDC',
    #'Pipeline H2 (g) / CH4 (g)':'(H2|CH4) \(g\)( submarine)? pipeline',
    #'Shipping':'(transport )?ship\s?(un)?(loading)?', # Match transport ship as well as (un-) loading operations
    "HVDC | Pipeline | Shipping": "((HVDC)|((H2|CH4) \(g\)( submarine)? pipeline)|((H2|CH4) \(g\) fill compressor station)|((transport )?ship\s?(un)?(loading)?))",
}

# Rename and aggregate columns
for c_new, c in columns_to_aggregate.items():
    tmp = df.filter(regex=c, axis=1)
    if tmp.empty:
        continue
    df = df.drop(tmp.columns, axis=1)
    df[c_new] = tmp.sum(axis=1)

In [ ]:
## Plotting
# Colormap selection/creation and iterator
number_of_colors = 13
cmap = plt.get_cmap("tab20c_r").reversed()
colors = itertools.cycle([cmap(i) for i in np.arange(number_of_colors)])

fig = plt.figure(figsize=(14, 6))
ax = plt.gca()

# Copy df and sort according to order of selected_escs list (to show in same order in plot)
tdf = df.copy().reindex(selected_escs, level=0)

legend_handles = []

for v in list(columns_to_aggregate.keys())[::-1]:  # backwards iteration necessary

    # New Skip? -> Reverse order relevant, i.e. from bottom to top in above enter those to skip
    if any(
        [
            v.startswith(s)
            for s in [
                "bla",
                #'Pipeline', 'HVDC', # same as shipping
                #'DAC', # same as LOHC chemical
                #'storage CH4','storage LOHC','storage hydrocarbon product', # same as storage NH3
                #'Ammonia synthesis','Methanol synthesis','Methanation','de-/hydrogenation LOHC' # same as FT synth.
            ]
        ]
    ):
        # Do not use a new color to reduce the color variety
        # the meaning of the color is inferable from the ESC
        # The list to skip is determined by the backwards order of the dict 'columns_to_aggregate'
        print(f"skipping new color for {v}")
    else:
        # New color for new technology
        c = next(colors)

    # Plotting a stacked grouped bar plot is stupid
    # we plot the bars and then plot over them with the lower values
    # need to sum() manually in each iteration after removing a column in the previous iteration
    # then create dedicated dataframes
    # reindex to preserve order in plot
    (
        (
            tdf.sum(axis=1)
            .to_frame(name="sum")
            .reset_index()
            .pivot(index="esc", columns="exporter", values="sum")
            .reindex(selected_escs)
        )
        / demand
    ).plot(
        kind="bar", ax=ax, color=[c]
    )  # pandas needs - for whatever reason - an iterable containing a tuple for RGBA (instead of just 'c'))
    tdf = tdf.drop(columns=v)

    # Add appropriate legend entry for technology
    legend_handles += [
        matplotlib.lines.Line2D(
            [],
            [],
            linestyle="None",
            markersize=10,
            markeredgecolor="black",
            marker="s",
            label=v,
            color=c,
        )
    ]

## Header for legend
# legend_handles = [
#    matplotlib.lines.Line2D(
#       [],
#        [],
#        color="None",
#        linestyle="None",
#        markersize=0,
#        label="Component",
#        marker=None,
#    )
# ] + legend_handles
ax.legend(handles=legend_handles, loc="upper left")
# ax.legend(handles=legend_handles, bbox_to_anchor=(1.05, 1), loc='center left')

# axis labels
ax.set_ylabel("Cost per MWh chemical delivered [EUR/MWh]")
ax.set_xlabel("ESC and exporter")
plt.xticks(rotation=0)  # Remove automatic rotation from pandas

# Add labels for exporters
width = ax.patches[0].get_width()
for x in ax.get_xticks():
    for i, exp in enumerate(selected_exps):
        ax.text(x + width * (i - 1.5), -20, exp, size="small")

ax.set_xticklabels(
    [
        "\n" + snakemake.config["plotting"]["esc_pretty_names"][esc.get_text()]
        for esc in ax.get_xticklabels()
    ]
)

# Minor gridlines
ax.grid(which="major", axis="y", visible=True, lw=2)
ax.grid(which="minor", axis="y", visible=True, alpha=0.5)
ax.minorticks_on()

# Save figure for all file types (PDF, PNG, ...)
for fp in snakemake.output["figures"]:
    plt.savefig(fp, dpi=snakemake.config["plotting"]["dpi"])